In [1]:
%load_ext autoreload
%autoreload 2

from Shared.shared import *
from Shared.specific_CNB_sim import *

import pandas as pd

sim_name = f"Dopri5_1k_z1"
sim_folder = f"sim_output/{sim_name}"
fig_folder = f"figures_local/{sim_name}"
Cl_folder = f"Shared/Cls"
nu_m_range = jnp.load(f"{sim_folder}/neutrino_massrange_eV.npy")
nu_m_picks = jnp.array([0.01, 0.05, 0.1, 0.2, 0.3])*Params.eV
simdata = SimData(sim_folder)

z_int_steps = jnp.load(f"{sim_folder}/z_int_steps.npy")
s_int_steps = jnp.load(f"{sim_folder}/s_int_steps.npy")
# print(z_int_steps)

# nu_vecs_FD = jnp.load(f"{sim_folder}/vectors_day1.npy")
# pix_dens_FD = jnp.load(f"{sim_folder}/pixel_densities_day1.npy")
# tot_dens_FD = jnp.load(f"{sim_folder}/total_densities_day1.npy")
# print(nu_vecs_FD[0,0,...])
# print(pix_dens_FD.shape)
# print(tot_dens_FD)

In [5]:
(46*nat.Glyr).convert(nat.Gpc)

<Quantity 14.10366411 Gpc>

In [8]:
cutoff = 1e-50
grad_sun = jnp.array([np.nan, 1e-40, 1e-52])
grad_sun = jnp.nan_to_num(
    grad_sun, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
# grad_sun = jnp.where(jnp.abs(grad_sun) < cutoff, 0.0, grad_sun)
print(grad_sun)
# grad_sun /= (Params.kpc/Params.s**2)
# print(grad_sun)

[0.e+00 1.e-40 1.e-52]


### CNB in solar region.

In [ ]:
ES_dist_av = 150.27e6*Params.km
nu_vel_max = 0.05*Params.c
nu_ES_time = ES_dist_av/nu_vel_max
time_unit = Params.s*60*60*24
print(nu_ES_time/time_unit)

### Neutrino mass hierarchies.

In [ ]:
### ====================================== ###
### Global loop over neutrino mass choices ###
### ====================================== ###

# We later always select the first of the 3 masses, to generate the Cl's.
# In that way we get Cl's precisely for first mass. Class still needs 3 masses.

nu_mass_set0 = Physics.neutrino_masses(
    m_lightest=0.00001, ordering='NO', args=Params())

# To get 0.01 eV
nu_mass_set1 = Physics.neutrino_masses(
    m_lightest=0.01, ordering='NO', args=Params())

# To get 0.05 eV
nu_mass_set2 = Physics.neutrino_masses(
    m_lightest=0.05, ordering='NO', args=Params())

# To get 0.1 eV
nu_mass_set3 = Physics.neutrino_masses(
    m_lightest=0.1, ordering='NO', args=Params())

# To get 0.2 eV
nu_mass_set4 = Physics.neutrino_masses(
    m_lightest=0.2, ordering='NO', args=Params())

# To get 0.3 eV
nu_mass_set5 = Physics.neutrino_masses(
    m_lightest=0.3, ordering='NO', args=Params())

nu_masses_sets = [
    nu_mass_set0.tolist(),
    nu_mass_set1.tolist(),
    nu_mass_set2.tolist(),
    nu_mass_set3.tolist(),
    nu_mass_set4.tolist(),
    nu_mass_set5.tolist(),
]

nu_masses = nu_masses_sets[0]
print(nu_masses)

# Always selecting first mass out of the 3
m_idx = 0
m_val = nu_masses[m_idx]

print(fr"*** Generating Cl for $m_\nu$={m_val} eV ***")

In [ ]:
@chex.dataclass
class Test:
    # Temperature of CMB
    T_CMB: float = 2.72548

    # N_eff parameter, using results of https://doi.org/10.1103/PhysRevD.108.L121301
    N_eff: float = 3.043

    # Temperature of CNB
    T_CNB = jnp.power(4/11, 1/3)*T_CMB * jnp.power(N_eff/3, 1/4)

    # Temperature of CNB
    def T_CNB_fct(N_eff=N_eff, T_CMB=T_CMB):
        return jnp.power(4/11, 1/3)*T_CMB * jnp.power(N_eff/3, 1/4)


print(Test.T_CNB_fct())
print(Test.T_CNB_fct(N_eff=3.0))

print("END")